In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from cs_bahtml.gospa import calculate_gospa_distance

In [47]:
df = pd.read_json('../data/parsed_util.json')

In [48]:
df

,map_name,entity_id,round,throwSeconds,grenade_type,throwerName,throwerTeam,throwerBuyType,opponentTeam,opponentBuyType,throwerSide,throwCords,landCords,starting_positions,dead_players,alive_players
0,de_inferno,68,8,24.890625,molotov,nyezin,paiN,full,Imperial,full,CT,"[1560.4774169922, 92.0494155884, 132.03125]","[1335.59375, 304.59375, 130.03125]","[[1560.4774169922, 92.0494155884, 132.03125], ...","[[341.7153320312, 1680.1881103516, 131.9216918...","[[1476.9392089844, -82.5447387695, 132.03125],..."
1,de_inferno,89,15,62.734375,smoke,decenty,Imperial,full,paiN,full,CT,"[1366.7225341797, 420.3508300781, 129.03125]","[125.9375, 773.8125, 81.5625]","[[1366.7225341797, 420.3508300781, 129.03125],...","[[645.6950073242, 2044.5159912109, 136.0312652...","[[2180.6687011719, 2401.2136230469, 187.604598..."
2,de_inferno,102,18,84.906250,molotov,biguzera,paiN,full,Imperial,full,TERRORIST,"[479.0244140625, 2016.96875, 136.0312042236]","[113.1875, 2556.625, 170.15625]","[[1265.96875, 365.9687194824, 129.5258636475],...",None,"[[876.0958862305, 61.2413101196, 204.03125], [..."
3,de_inferno,114,18,61.250000,smoke,noway,Imperial,full,paiN,full,CT,"[930.5641479492, 2731.2006835938, 133.7768249512]","[739.90625, 2329.8125, 138.03125]","[[1213.0803222656, 169.05128479, 206.116531372...","[[1213.0803222656, 169.05128479, 206.116531372...","[[2553.8901367188, 1329.1119384766, 160.03125]..."
4,de_inferno,144,17,15.062500,smoke,nyezin,paiN,full,Imperial,full,TERRORIST,"[872.96875, 43.9686965942, 135.2767944336]","[1400.125, 1021.1875, 156.34375]","[[872.96875, 43.9686965942, 135.2767944336], [...",None,"[[1207.1674804688, 565.2262573242, 122.9673080..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32182,de_inferno,1520,4,35.468750,smoke,HEN1-,Imperial,force,paiN Gaming,full,CT,"[1508.7307128906, 1042.8937988281, 156.861541748]","[1230.3125, 504.125, 117.5625]","[[1850.9664306641, -173.3381347656, 260.03125]...","[[665.3968505859, 2720.8930664062, 146.2247467...","[[864.4584350586, -346.8254394531, 89.03125], ..."
32183,de_inferno,1528,4,47.781250,smoke,VINI,Imperial,force,paiN Gaming,full,CT,"[682.7188720703, 3555.591796875, 141.03125]","[229.53125, 2853.21875, 163.53125]","[[963.9238891602, -647.1146240234, 97.54673767...","[[665.3968505859, 2720.8930664062, 146.2247467...","[[288.4042053223, 685.6383056641, 26.943048477..."
32184,de_inferno,1531,4,49.562500,molotov,biguzera,paiN Gaming,full,Imperial,force,TERRORIST,"[329.9813842773, 3074.4243164062, 168.4594116211]","[726.0, 3377.96875, 132.5]","[[641.1748657227, 2456.2253417969, 189.03125],...","[[314.4516906738, 2590.59375, 164.03125]]","[[738.9270019531, 2197.4838867188, 136.0312957..."
32185,de_inferno,1533,4,57.875000,smoke,kauez,paiN Gaming,full,Imperial,force,TERRORIST,"[755.7681884766, 2858.6398925781, 139.58152771]","[1007.46875, 2697.8125, 132.21875]","[[721.9446411133, 2266.3999023438, 136.0314636...","[[314.4516906738, 2590.59375, 164.03125]]","[[152.0003356934, 1231.6242675781, 94.83277130..."


In [51]:
from scipy.spatial.distance import pdist, squareform, cdist
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import numpy as np
# Define a function to calculate Euclidean distance
def calculate_distance(cords):
    return pdist(cords, 'euclidean')
def calculate_equal_distance(series):
    x = np.asarray(series)
    return squareform((x[:, None] != x[None, :]).astype(int))*1e9

def calculate_clusters(df, threshold=200):
    # Create distance matrices for throwCords and landCords
    throwCords = df['throwCords'].tolist()
    landCords = df['landCords'].tolist()
    throwCords_dist = calculate_distance(throwCords)
    landCords_dist = calculate_distance(landCords)

    #map_name_dist = calculate_equal_distance(df['map_name'])
    #throwerSide_dist = calculate_equal_distance(df['throwerSide'])
    #grenade_type_dist = calculate_equal_distance(df['grenade_type'])
    #max_grouping_dist = np.maximum.reduce([map_name_dist, throwerSide_dist, grenade_type_dist])

    
    # Use the maximum distance between throwCords and landCords
    #max_dist = np.maximum.reduce([throwCords_dist, landCords_dist, max_grouping_dist])
    max_dist = np.maximum(throwCords_dist, landCords_dist)

    # Use this distance matrix in hierarchical clustering
    clusters = linkage(max_dist, method='complete')
    clusters = fcluster(clusters, threshold, criterion='distance')
    out_df = pd.DataFrame(
                    {"idx": df.index, 
                     "cluster": clusters})
    return out_df


In [64]:
clusters = df.groupby(["map_name","throwerSide", "grenade_type"])[['throwCords','landCords']].apply(calculate_clusters, threshold=200).set_index(["idx"])
df["cluster"]=None
df.update(clusters)
clusters_sorted = df.cluster.value_counts().index.tolist()

In [54]:
df

,map_name,entity_id,round,throwSeconds,grenade_type,throwerName,throwerTeam,throwerBuyType,opponentTeam,opponentBuyType,throwerSide,throwCords,landCords,starting_positions,dead_players,alive_players,cluster
0,de_inferno,68,8,24.890625,molotov,nyezin,paiN,full,Imperial,full,CT,"[1560.4774169922, 92.0494155884, 132.03125]","[1335.59375, 304.59375, 130.03125]","[[1560.4774169922, 92.0494155884, 132.03125], ...","[[341.7153320312, 1680.1881103516, 131.9216918...","[[1476.9392089844, -82.5447387695, 132.03125],...",24
1,de_inferno,89,15,62.734375,smoke,decenty,Imperial,full,paiN,full,CT,"[1366.7225341797, 420.3508300781, 129.03125]","[125.9375, 773.8125, 81.5625]","[[1366.7225341797, 420.3508300781, 129.03125],...","[[645.6950073242, 2044.5159912109, 136.0312652...","[[2180.6687011719, 2401.2136230469, 187.604598...",14
2,de_inferno,102,18,84.906250,molotov,biguzera,paiN,full,Imperial,full,TERRORIST,"[479.0244140625, 2016.96875, 136.0312042236]","[113.1875, 2556.625, 170.15625]","[[1265.96875, 365.9687194824, 129.5258636475],...",None,"[[876.0958862305, 61.2413101196, 204.03125], [...",5
3,de_inferno,114,18,61.250000,smoke,noway,Imperial,full,paiN,full,CT,"[930.5641479492, 2731.2006835938, 133.7768249512]","[739.90625, 2329.8125, 138.03125]","[[1213.0803222656, 169.05128479, 206.116531372...","[[1213.0803222656, 169.05128479, 206.116531372...","[[2553.8901367188, 1329.1119384766, 160.03125]...",23
4,de_inferno,144,17,15.062500,smoke,nyezin,paiN,full,Imperial,full,TERRORIST,"[872.96875, 43.9686965942, 135.2767944336]","[1400.125, 1021.1875, 156.34375]","[[872.96875, 43.9686965942, 135.2767944336], [...",None,"[[1207.1674804688, 565.2262573242, 122.9673080...",7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32182,de_inferno,1520,4,35.468750,smoke,HEN1-,Imperial,force,paiN Gaming,full,CT,"[1508.7307128906, 1042.8937988281, 156.861541748]","[1230.3125, 504.125, 117.5625]","[[1850.9664306641, -173.3381347656, 260.03125]...","[[665.3968505859, 2720.8930664062, 146.2247467...","[[864.4584350586, -346.8254394531, 89.03125], ...",16
32183,de_inferno,1528,4,47.781250,smoke,VINI,Imperial,force,paiN Gaming,full,CT,"[682.7188720703, 3555.591796875, 141.03125]","[229.53125, 2853.21875, 163.53125]","[[963.9238891602, -647.1146240234, 97.54673767...","[[665.3968505859, 2720.8930664062, 146.2247467...","[[288.4042053223, 685.6383056641, 26.943048477...",25
32184,de_inferno,1531,4,49.562500,molotov,biguzera,paiN Gaming,full,Imperial,force,TERRORIST,"[329.9813842773, 3074.4243164062, 168.4594116211]","[726.0, 3377.96875, 132.5]","[[641.1748657227, 2456.2253417969, 189.03125],...","[[314.4516906738, 2590.59375, 164.03125]]","[[738.9270019531, 2197.4838867188, 136.0312957...",2
32185,de_inferno,1533,4,57.875000,smoke,kauez,paiN Gaming,full,Imperial,force,TERRORIST,"[755.7681884766, 2858.6398925781, 139.58152771]","[1007.46875, 2697.8125, 132.21875]","[[721.9446411133, 2266.3999023438, 136.0314636...","[[314.4516906738, 2590.59375, 164.03125]]","[[152.0003356934, 1231.6242675781, 94.83277130...",26


In [55]:
from cs_bahtml.plotting import add_smoke_wall, add_players
from cs_bahtml.radars import RADARS
def plot_cluster(df, cluster):
    plot_df = df[df.cluster == cluster]
    map_name = plot_df.map_name.iloc[0]
    assert plot_df.map_name.nunique() == 1
    fig = RADARS[map_name].copy()
    for idx, row in plot_df.iterrows():
        fig = add_smoke_wall(fig, [row['landCords']], map_name, type = row['grenade_type'])
        fig = add_players(fig, [row['throwCords']], map_name, type = 'thrower')
    fig.show()

In [66]:
map_df = df.query("map_name == 'de_ancient' & throwerSide == 'TERRORIST' & grenade_type == 'smoke'")
clusters_sorted = map_df.cluster.value_counts().index.tolist()

for cluster in clusters_sorted[:10]: 
    print(cluster)
    
    plot_cluster(map_df, cluster)

169
225
172
98
216
93
155
180
206
142


In [67]:
df.to_json('../data/parsed_clusters.json')